---
# **PHARMACY DASHBOARD PREREQUISITES**
---

---
#### IMPORT LIBRARIES


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

---
#### LOAD DATASET


In [2]:
customer_df = pd.read_csv("customer_data.csv")
product_df = pd.read_csv("product_master.csv")
transaction_df = pd.read_csv("transaction_data.csv")

In [3]:
# customer data
customer_df.head()

,CustomerID,CustomerType,AgeGroup,Gender,Location,DateOfFirstPurchase,LastPurchaseDate
0,C00000,Returning,18-25,Male,Lake Richardberg,2024-02-14,2024-06-07
1,C00001,Returning,18-25,Other,Kellychester,2023-08-02,2024-07-12
2,C00002,New,18-25,Female,Longbury,2024-03-29,2024-08-04
3,C00003,Returning,<18,Other,Kellychester,2024-03-22,2024-04-08
4,C00004,Returning,26-40,Other,Brownberg,2024-01-05,2025-02-16


In [4]:
# product master
product_df.head()
# NOTE: Reorder Level = Average Daily Usage × Lead Time (in days)

,ProductID,ProductName,Category,CostPrice,StockQuantity,ReorderLevel,ExpiryDate,SellingPrice
0,P0000,Product rwHC,Wellness,352.10,313,19,2027-12-14,462.06
1,P0001,Product FUtF,Equipment,272.69,459,11,2026-06-10,370.96
2,P0002,Product KRjS,Prescription,161.67,463,25,2027-06-11,219.52
3,P0003,Product Ixmd,Wellness,408.76,421,46,2027-04-12,541.41
4,P0004,Product jnwE,Wellness,345.52,126,30,2026-11-22,487.45


In [5]:
# transaction data
transaction_df.head()

,TransactionID,Date,ProductID,Quantity,DiscountPercent,PaymentMethod,UnitPrice,TotalAmount,FinalAmount,CustomerID
0,T0000000,2023-11-09,P0927,8,15,UPI,643.38,5147.04,4374.98,C28772
1,T0000001,2024-02-14,P0615,8,30,Card,425.06,3400.48,2380.34,NaN
2,T0000002,2023-10-24,P0910,4,0,UPI,149.23,596.92,596.92,C39200
3,T0000003,2024-10-20,P0753,1,10,Cash,80.37,80.37,72.33,C11128
4,T0000004,2025-02-04,P0333,9,30,Card,127.81,1150.29,805.20,C37550


Merging datasets

In [6]:
c_t_df = pd.merge(customer_df, transaction_df, on="CustomerID")
final_df = pd.merge(c_t_df, product_df, on="ProductID")
final_df.head()

,CustomerID,CustomerType,AgeGroup,Gender,Location,DateOfFirstPurchase,LastPurchaseDate,TransactionID,Date,ProductID,...,UnitPrice,TotalAmount,FinalAmount,ProductName,Category,CostPrice,StockQuantity,ReorderLevel,ExpiryDate,SellingPrice
0,C00000,Returning,18-25,Male,Lake Richardberg,2024-02-14,2024-06-07,T0020173,2025-02-14,P0064,...,260.28,780.84,780.84,Product Cohk,Wellness,181.79,99,26,2026-11-20,260.28
1,C00001,Returning,18-25,Other,Kellychester,2023-08-02,2024-07-12,T0022207,2025-05-05,P0805,...,510.17,4081.36,3469.16,Product jMJW,Equipment,436.77,319,42,2026-08-11,510.17
2,C00001,Returning,18-25,Other,Kellychester,2023-08-02,2024-07-12,T0033667,2024-11-08,P0394,...,490.52,4905.20,3678.90,Product zsBG,Wellness,337.36,343,43,2026-06-28,490.52
3,C00002,New,18-25,Female,Longbury,2024-03-29,2024-08-04,T0019925,2024-12-01,P0237,...,514.67,5146.70,4374.70,Product ipVZ,OTC,410.13,451,31,2026-04-03,514.67
4,C00002,New,18-25,Female,Longbury,2024-03-29,2024-08-04,T0025197,2024-08-04,P0663,...,533.44,2667.20,2533.84,Product usEA,OTC,374.32,423,36,2028-02-28,533.44


In [7]:
# create a copy of dataset
ml_df = final_df.copy()

In [8]:
final_df.shape

(90000, 23)

In [9]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           90000 non-null  object 
 1   CustomerType         90000 non-null  object 
 2   AgeGroup             90000 non-null  object 
 3   Gender               90000 non-null  object 
 4   Location             90000 non-null  object 
 5   DateOfFirstPurchase  90000 non-null  object 
 6   LastPurchaseDate     90000 non-null  object 
 7   TransactionID        90000 non-null  object 
 8   Date                 90000 non-null  object 
 9   ProductID            90000 non-null  object 
 10  Quantity             90000 non-null  int64  
 11  DiscountPercent      90000 non-null  int64  
 12  PaymentMethod        90000 non-null  object 
 13  UnitPrice            90000 non-null  float64
 14  TotalAmount          90000 non-null  float64
 15  FinalAmount          90000 non-null 

#### Automated cleaning

In [10]:
# GET datetime columns
datetime_list = []

# take each column
for col in final_df.columns:
    # Drop nulls and convert to string
    non_null_data = final_df[col].dropna().astype(str)
    
    # Sample min(20, available rows)
    sample_size = min(20, len(non_null_data))
    sample_rows = non_null_data.sample(sample_size)
    
    count = 0

    if pd.api.types.is_object_dtype(final_df[col]):
        for item in sample_rows:
            try:
                pd.to_datetime(item)
                count += 1
            except Exception:
                continue

        if count / sample_size >= 0.8:
            datetime_list.append(col)

datetime_list


['DateOfFirstPurchase', 'LastPurchaseDate', 'Date', 'ExpiryDate']

In [11]:
# convert datetime_list to datetime
for dt in datetime_list:
    final_df[dt] = pd.to_datetime(final_df[dt])

final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   CustomerID           90000 non-null  object        
 1   CustomerType         90000 non-null  object        
 2   AgeGroup             90000 non-null  object        
 3   Gender               90000 non-null  object        
 4   Location             90000 non-null  object        
 5   DateOfFirstPurchase  90000 non-null  datetime64[ns]
 6   LastPurchaseDate     90000 non-null  datetime64[ns]
 7   TransactionID        90000 non-null  object        
 8   Date                 90000 non-null  datetime64[ns]
 9   ProductID            90000 non-null  object        
 10  Quantity             90000 non-null  int64         
 11  DiscountPercent      90000 non-null  int64         
 12  PaymentMethod        90000 non-null  object        
 13  UnitPrice            90000 non-

In [12]:
# id column
id_col = [col for col in final_df.columns if "ID" in col]
id_col

['CustomerID', 'TransactionID', 'ProductID']

In [13]:
# filter out the id column
num_cat_col = [col for col in final_df.columns if 'ID' not in col]
num_cat_col

['CustomerType',
 'AgeGroup',
 'Gender',
 'Location',
 'DateOfFirstPurchase',
 'LastPurchaseDate',
 'Date',
 'Quantity',
 'DiscountPercent',
 'PaymentMethod',
 'UnitPrice',
 'TotalAmount',
 'FinalAmount',
 'ProductName',
 'Category',
 'CostPrice',
 'StockQuantity',
 'ReorderLevel',
 'ExpiryDate',
 'SellingPrice']

In [14]:
# cleaning the data (imputing the null values)
num_col = final_df[num_cat_col].select_dtypes(include=['number']).columns.tolist()
num_col

['Quantity',
 'DiscountPercent',
 'UnitPrice',
 'TotalAmount',
 'FinalAmount',
 'CostPrice',
 'StockQuantity',
 'ReorderLevel',
 'SellingPrice']

In [15]:
# categorical column
cat_col = final_df[num_cat_col].select_dtypes(include=['object', 'category']).columns.tolist()
cat_col

['CustomerType',
 'AgeGroup',
 'Gender',
 'Location',
 'PaymentMethod',
 'ProductName',
 'Category']

In [16]:
# numerical
numeric_but_cat = [col for col in num_col if final_df[col].nunique() < 10]
numeric_but_cat

# extend the list
for col in numeric_but_cat:
    final_df[col] = final_df[col].astype('object')
    cat_col.append(col)
    num_col.remove(col)

print(cat_col)
print(num_col)

['CustomerType', 'AgeGroup', 'Gender', 'Location', 'PaymentMethod', 'ProductName', 'Category', 'DiscountPercent']
['Quantity', 'UnitPrice', 'TotalAmount', 'FinalAmount', 'CostPrice', 'StockQuantity', 'ReorderLevel', 'SellingPrice']


#### Data cleaning for KPI analysis

In [17]:
# impute for kpi
# impute for numerical columns
print(num_col)
print(cat_col)


['Quantity', 'UnitPrice', 'TotalAmount', 'FinalAmount', 'CostPrice', 'StockQuantity', 'ReorderLevel', 'SellingPrice']
['CustomerType', 'AgeGroup', 'Gender', 'Location', 'PaymentMethod', 'ProductName', 'Category', 'DiscountPercent']


In [18]:
# numerical transformation
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

numerical_transformer

Pipeline(steps=[('imputer', SimpleImputer())])

In [19]:
# categorical transformation
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

categorical_transformer

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent'))])

In [20]:
# full column transformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, num_col),
    ('cat', categorical_transformer, cat_col)
])

preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['Quantity', 'UnitPrice', 'TotalAmount',
                                  'FinalAmount', 'CostPrice', 'StockQuantity',
                                  'ReorderLevel', 'SellingPrice']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['CustomerType', 'AgeGroup', 'Gender',
                                  'Location', 'PaymentMethod', 'ProductName',
                                  'Category', 'DiscountPercent'])])

In [21]:
# fit and transform the data
processed_array = preprocessor.fit_transform(final_df)
processed_array

array([[3.0, 260.28, 780.8399999999999, ..., 'Product Cohk', 'Wellness',
        0],
       [8.0, 510.17, 4081.36, ..., 'Product jMJW', 'Equipment', 15],
       [10.0, 490.52, 4905.2, ..., 'Product zsBG', 'Wellness', 25],
       ...,
       [7.0, 477.98, 3345.86, ..., 'Product bRXD', 'Prescription', 20],
       [6.0, 381.72, 2290.32, ..., 'Product xGOA', 'Equipment', 25],
       [4.0, 625.4, 2501.6, ..., 'Product bKnH', 'OTC', 5]],
      shape=(90000, 16), dtype=object)

## **USE it for KPI**
Basic columns: 
* 'Date' *
* 'CostPrice' *
* 'SellingPrice' * 
* 'Quantity' *
* 'DiscountPercent' * 
* 'FinalAmount' 
* 'ProductName' *
* 'Category' *
* 'CustomerID' *
* 'ProductID' *
* 'TransactionID' *

Created Imputed Dataframe For KPI

In [22]:
# convert to dataframe
processed_df = pd.DataFrame(processed_array, columns=num_col + cat_col)
processed_df.head()

,Quantity,UnitPrice,TotalAmount,FinalAmount,CostPrice,StockQuantity,ReorderLevel,SellingPrice,CustomerType,AgeGroup,Gender,Location,PaymentMethod,ProductName,Category,DiscountPercent
0,3.0,260.28,780.84,780.84,181.79,99.0,26.0,260.28,Returning,18-25,Male,Lake Richardberg,UPI,Product Cohk,Wellness,0
1,8.0,510.17,4081.36,3469.16,436.77,319.0,42.0,510.17,Returning,18-25,Other,Kellychester,Wallet,Product jMJW,Equipment,15
2,10.0,490.52,4905.2,3678.9,337.36,343.0,43.0,490.52,Returning,18-25,Other,Kellychester,Wallet,Product zsBG,Wellness,25
3,10.0,514.67,5146.7,4374.7,410.13,451.0,31.0,514.67,New,18-25,Female,Longbury,Cash,Product ipVZ,OTC,15
4,5.0,533.44,2667.2,2533.84,374.32,423.0,36.0,533.44,New,18-25,Female,Longbury,Wallet,Product usEA,OTC,5


In [23]:
# add the id columns
processed_final_df = pd.concat([final_df[id_col].reset_index(drop=True), processed_df], axis=1)
processed_final_df = pd.concat([processed_final_df, final_df[datetime_list].reset_index(drop=True)], axis=1)
processed_final_df.head()

,CustomerID,TransactionID,ProductID,Quantity,UnitPrice,TotalAmount,FinalAmount,CostPrice,StockQuantity,ReorderLevel,...,Gender,Location,PaymentMethod,ProductName,Category,DiscountPercent,DateOfFirstPurchase,LastPurchaseDate,Date,ExpiryDate
0,C00000,T0020173,P0064,3.0,260.28,780.84,780.84,181.79,99.0,26.0,...,Male,Lake Richardberg,UPI,Product Cohk,Wellness,0,2024-02-14,2024-06-07,2025-02-14,2026-11-20
1,C00001,T0022207,P0805,8.0,510.17,4081.36,3469.16,436.77,319.0,42.0,...,Other,Kellychester,Wallet,Product jMJW,Equipment,15,2023-08-02,2024-07-12,2025-05-05,2026-08-11
2,C00001,T0033667,P0394,10.0,490.52,4905.2,3678.9,337.36,343.0,43.0,...,Other,Kellychester,Wallet,Product zsBG,Wellness,25,2023-08-02,2024-07-12,2024-11-08,2026-06-28
3,C00002,T0019925,P0237,10.0,514.67,5146.7,4374.7,410.13,451.0,31.0,...,Female,Longbury,Cash,Product ipVZ,OTC,15,2024-03-29,2024-08-04,2024-12-01,2026-04-03
4,C00002,T0025197,P0663,5.0,533.44,2667.2,2533.84,374.32,423.0,36.0,...,Female,Longbury,Wallet,Product usEA,OTC,5,2024-03-29,2024-08-04,2024-08-04,2028-02-28


In [24]:
processed_final_df.columns

Index(['CustomerID', 'TransactionID', 'ProductID', 'Quantity', 'UnitPrice',
       'TotalAmount', 'FinalAmount', 'CostPrice', 'StockQuantity',
       'ReorderLevel', 'SellingPrice', 'CustomerType', 'AgeGroup', 'Gender',
       'Location', 'PaymentMethod', 'ProductName', 'Category',
       'DiscountPercent', 'DateOfFirstPurchase', 'LastPurchaseDate', 'Date',
       'ExpiryDate'],
      dtype='object')

In [25]:
processed_final_df.head()

,CustomerID,TransactionID,ProductID,Quantity,UnitPrice,TotalAmount,FinalAmount,CostPrice,StockQuantity,ReorderLevel,...,Gender,Location,PaymentMethod,ProductName,Category,DiscountPercent,DateOfFirstPurchase,LastPurchaseDate,Date,ExpiryDate
0,C00000,T0020173,P0064,3.0,260.28,780.84,780.84,181.79,99.0,26.0,...,Male,Lake Richardberg,UPI,Product Cohk,Wellness,0,2024-02-14,2024-06-07,2025-02-14,2026-11-20
1,C00001,T0022207,P0805,8.0,510.17,4081.36,3469.16,436.77,319.0,42.0,...,Other,Kellychester,Wallet,Product jMJW,Equipment,15,2023-08-02,2024-07-12,2025-05-05,2026-08-11
2,C00001,T0033667,P0394,10.0,490.52,4905.2,3678.9,337.36,343.0,43.0,...,Other,Kellychester,Wallet,Product zsBG,Wellness,25,2023-08-02,2024-07-12,2024-11-08,2026-06-28
3,C00002,T0019925,P0237,10.0,514.67,5146.7,4374.7,410.13,451.0,31.0,...,Female,Longbury,Cash,Product ipVZ,OTC,15,2024-03-29,2024-08-04,2024-12-01,2026-04-03
4,C00002,T0025197,P0663,5.0,533.44,2667.2,2533.84,374.32,423.0,36.0,...,Female,Longbury,Wallet,Product usEA,OTC,5,2024-03-29,2024-08-04,2024-08-04,2028-02-28


In [26]:
processed_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   CustomerID           90000 non-null  object        
 1   TransactionID        90000 non-null  object        
 2   ProductID            90000 non-null  object        
 3   Quantity             90000 non-null  object        
 4   UnitPrice            90000 non-null  object        
 5   TotalAmount          90000 non-null  object        
 6   FinalAmount          90000 non-null  object        
 7   CostPrice            90000 non-null  object        
 8   StockQuantity        90000 non-null  object        
 9   ReorderLevel         90000 non-null  object        
 10  SellingPrice         90000 non-null  object        
 11  CustomerType         90000 non-null  object        
 12  AgeGroup             90000 non-null  object        
 13  Gender               90000 non-

#### **Create Date Range**

In [27]:
start_date_tuple = pd.to_datetime("2025-1-1")
end_date_tuple = pd.to_datetime("2026-5-1")
start_date = pd.to_datetime(start_date_tuple)
end_date = pd.to_datetime(end_date_tuple)
print(start_date)
print(end_date)

2025-01-01 00:00:00
2026-05-01 00:00:00


In [28]:
mask = ((processed_final_df['Date'] >= start_date) & (processed_final_df['Date'] <= end_date))
filtered_df = processed_final_df[mask]

#### Total Sales

In [29]:
# 'FinalAmount', 'SellingPrice', 'Discount', 'Quantity'
if 'FinalAmount' in filtered_df.columns:
    total_sales = filtered_df['FinalAmount'].sum()
    print(f"Total Sales: {total_sales}")
elif 'SellingPrice' in filtered_df.columns and 'Discount' in filtered_df.columns:
    # Calculate total sales based on SellingPrice and Discount
    total_sales = (filtered_df['Quantity'] * filtered_df['SellingPrice']) - (filtered_df['Quantity'] * filtered_df['SellingPrice'] * filtered_df['Discount'] / 100).sum()
    
    print(f"Total sales: {total_sales}")

Total Sales: 24048239.37999983


#### Gross Profit Margin

In [30]:
# 'FinalAmount', 'CostPrice', 'Quantity'
gross_profit = filtered_df['FinalAmount'].sum() - (filtered_df['Quantity'] * filtered_df['CostPrice']).sum()
print(f"Gross Profit: {gross_profit}")
gross_profit_margin = gross_profit / filtered_df['FinalAmount'].sum() * 100

print(f"Gross Profit Margin (in percentage): {gross_profit_margin:.2f}%")

Gross Profit: 2450689.5199997984
Gross Profit Margin (in percentage): 10.19%


#### Total Customers

In [31]:
# 'CustomerID
total_customers = filtered_df['CustomerID'].nunique()

print(f"Total Customers: {total_customers}")

Total Customers: 13392


#### Customer Frequency

In [32]:
# 'TransactionID', 'CustomerID'
# customer_frequency = Total Orders / Unique Customers
customer_frequency = filtered_df['TransactionID'].nunique() / filtered_df['CustomerID'].nunique()

print(f"Customer Frequency: {customer_frequency:.2f} orders/customer")

Customer Frequency: 1.16 orders/customer


#### Average Order Value

In [33]:
# 'FinalAmount', 'TransactionID'
average_order_value = processed_final_df['FinalAmount'].sum() / processed_final_df['TransactionID'].nunique()
print(f"Average Order Value: {average_order_value:.2f}")

Average Order Value: 1554.96


## **GRAPHICAL ANALYTICS**

TRENDLINE OF TOTAL SALES

In [34]:
# create filtered dataframes for different time periods
# get today's time
today = pd.Timestamp.today()

if "Date" not in processed_final_df.columns:
    print("Missing 'Date' column. Graphs may not render correctly.")
    processed_final_df["Date"] = pd.NaT

processed_final_df["Date"] = pd.to_datetime(processed_final_df["Date"], errors='coerce')

# Filter for current year
yearly_data = processed_final_df[processed_final_df["Date"].dt.year == today.year]

# Filter for current quarter
quarterly_data = yearly_data[
    yearly_data["Date"].dt.quarter == today.quarter
    ]


# Filter for current month
monthly_data = quarterly_data[quarterly_data["Date"].dt.month == today.month]

# Filter for current week (by weekday) — this line is incorrect in your version
# Fix: Use `.dt.weekday` instead of `.dt.month` for comparison with `self.today.weekday()`
start_of_week = today - pd.Timedelta(days=today.weekday())
end_of_week = start_of_week + pd.Timedelta(days=6)

weekly_data = processed_final_df[
    (processed_final_df["Date"] >= start_of_week) &
    (processed_final_df["Date"] <= end_of_week)
    ]

# Filter for current day
daily_data = monthly_data[monthly_data["Date"].dt.day == today.day]

In [35]:
daily_data['Date']

Series([], Name: Date, dtype: datetime64[ns])

In [36]:
weekly_data['Date']

Series([], Name: Date, dtype: datetime64[ns])

In [37]:
monthly_data['Date']

1       2025-05-05
352     2025-05-04
472     2025-05-01
565     2025-05-02
598     2025-05-01
           ...    
89832   2025-05-06
89858   2025-05-01
89903   2025-05-04
89932   2025-05-04
89980   2025-05-06
Name: Date, Length: 877, dtype: datetime64[ns]

In [38]:
quarterly_data['Date']

1       2025-05-05
20      2025-04-27
31      2025-04-19
67      2025-04-25
83      2025-04-08
           ...    
89967   2025-04-25
89972   2025-04-06
89980   2025-05-06
89990   2025-04-12
89995   2025-04-14
Name: Date, Length: 4578, dtype: datetime64[ns]

In [39]:
yearly_data['Date']

0       2025-02-14
1       2025-05-05
7       2025-03-18
9       2025-03-20
17      2025-01-02
           ...    
89974   2025-01-21
89979   2025-03-23
89980   2025-05-06
89990   2025-04-12
89995   2025-04-14
Name: Date, Length: 15539, dtype: datetime64[ns]

In [40]:
# trend analysis
monthly_totals = yearly_data.groupby(yearly_data['Date'].dt.to_period('M'))['FinalAmount'].sum().reset_index()

monthly_totals['Date'] = monthly_totals['Date'].dt.to_timestamp()

fig = px.line(
    monthly_totals, 
    x='Date', 
    y='FinalAmount', 
    title="This Year's Monthly Sales",
    labels={'FinalAmount': 'Total Sales'},
    markers=True
)
fig.show()

In [41]:
# top 10 best selling products

if yearly_data is None or "ProductName" not in yearly_data.columns:
    print("Missing 'ProductName' column for bar graph.")


top_products = yearly_data["ProductName"].value_counts().sort_values(ascending=False).head(20)
df_top = top_products.reset_index()
df_top.columns = ["ProductName", "SalesCount"]

fig = px.bar(
    df_top,
    x="ProductName",
    y="SalesCount",
    color="ProductName",  # Categorical coloring
    title="THIS YEAR'S TOP 20 BEST SELLING PRODUCTS",
    color_discrete_sequence=px.colors.qualitative.Plotly  # Use a vibrant palette
)

fig.show()

In [42]:
# profit margin by category
category_col = "Category"
revenue_col = "FinalAmount"
cost_price_col = "CostPrice"
quantity_col = "Quantity"

required_cols = [category_col, revenue_col, cost_price_col, quantity_col]
missing = [col for col in required_cols if col not in yearly_data.columns]
if missing:
    print(f"Missing columns for profit margin chart: {', '.join(missing)}")


yearly_data.loc[:, "Cost"] = yearly_data[cost_price_col] * yearly_data[quantity_col]
if "FinalAmount" in yearly_data.columns:
    yearly_data.loc[:, "Profit"] = yearly_data[revenue_col] - yearly_data[cost_price_col] * yearly_data[quantity_col]
elif ("SellingPrice" in yearly_data.columns) and ("Quantity" in yearly_data.columns):
    yearly_data.loc[:, "Profit"] = (yearly_data["SellingPrice"] * yearly_data["Quantity"]) - yearly_data["Cost"]

grouped = yearly_data.groupby(category_col).agg(
    Total_Profit=("Profit", "sum"),
    Total_Revenue=(revenue_col, "sum")
).reset_index()

grouped["Profit_Margin (%)"] = round((grouped["Total_Profit"] / grouped["Total_Revenue"]) * 100, 2)
grouped = grouped.sort_values("Profit_Margin (%)", ascending=False)

fig = px.bar(
    grouped,
    x=category_col,
    y="Profit_Margin (%)",
    color="Profit_Margin (%)",
    title="YEAR'S PROFIT MARGIN BY CATEGORY",
    color_continuous_scale="Viridis"
)

fig.show()

C:\Users\Dell\AppData\Local\Temp\ipykernel_2816\278046671.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Dell\AppData\Local\Temp\ipykernel_2816\278046671.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## **INVENTORY AGING TABLE (TOP10)**

In [43]:
if "ProductID" not in processed_final_df.columns or "Date" not in processed_final_df.columns:
    print("Missing 'ProductID' or 'Date' column for inventory aging.")


# Step 1: Get latest transaction date per ProductID
last_transaction = processed_final_df.groupby('ProductID')['Date'].max().reset_index()
last_transaction.columns = ['ProductID', 'LastTransactionDate']

# Step 2: Calculate inventory age
last_transaction['InventoryAge (days)'] = (today - last_transaction['LastTransactionDate']).dt.days

# Step 3: Get product names (unique)
product_names = processed_final_df[['ProductID', 'ProductName']].drop_duplicates()

# Step 4: Merge to bring in ProductName
last_transaction = pd.merge(last_transaction, product_names, on='ProductID', how='left')

# Step 5: Sort by age
last_transaction = last_transaction.sort_values(by='InventoryAge (days)', ascending=False)

last_transaction[['ProductID', 'ProductName', 'LastTransactionDate', 'InventoryAge (days)']]

,ProductID,ProductName,LastTransactionDate,InventoryAge (days)
509,P0509,Product dZNm,2025-03-19,68
996,P0996,Product HuRr,2025-03-23,64
836,P0836,Product uYUO,2025-03-25,62
39,P0039,Product MPjf,2025-03-26,61
978,P0978,Product idgD,2025-03-27,60
...,...,...,...,...
45,P0045,Product trKr,2025-05-07,19
961,P0961,Product kQSQ,2025-05-07,19
25,P0025,Product IcAd,2025-05-07,19
12,P0012,Product nDek,2025-05-07,19


## **MACHINE LEARNING**

In [44]:
# for ml_df
# GET datetime columns
datetime_list = []

# take each column
for col in ml_df.columns:
    # Drop nulls and convert to string
    non_null_data = ml_df[col].dropna().astype(str)

    # Sample min(20, available rows)
    sample_size = min(20, len(non_null_data))
    sample_rows = non_null_data.sample(sample_size)
    
    count = 0

    if pd.api.types.is_object_dtype(ml_df[col]):
        for item in sample_rows:
            try:
                pd.to_datetime(item)
                count += 1
            except Exception:
                continue

        if count / sample_size >= 0.8:
            datetime_list.append(col)

datetime_list


['DateOfFirstPurchase', 'LastPurchaseDate', 'Date', 'ExpiryDate']

In [45]:
# convert to datetime
for dt in datetime_list:
    ml_df[dt] = pd.to_datetime(ml_df[dt])
ml_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   CustomerID           90000 non-null  object        
 1   CustomerType         90000 non-null  object        
 2   AgeGroup             90000 non-null  object        
 3   Gender               90000 non-null  object        
 4   Location             90000 non-null  object        
 5   DateOfFirstPurchase  90000 non-null  datetime64[ns]
 6   LastPurchaseDate     90000 non-null  datetime64[ns]
 7   TransactionID        90000 non-null  object        
 8   Date                 90000 non-null  datetime64[ns]
 9   ProductID            90000 non-null  object        
 10  Quantity             90000 non-null  int64         
 11  DiscountPercent      90000 non-null  int64         
 12  PaymentMethod        90000 non-null  object        
 13  UnitPrice            90000 non-

In [46]:
# now to prepare the data for machine learning using simple imputer, one hot encoder and standard scaler
# removing the id columns
print(id_col)
print(cat_col)
print(num_col)

# remove id columns from the list
ml_cleaned_df = ml_df.drop(columns=id_col).copy()
# remove datetime columns from the list
ml_final_df = ml_cleaned_df.drop(columns=datetime_list).copy()

ml_model_df = ml_final_df[num_col].copy()
ml_model_df.info()

['CustomerID', 'TransactionID', 'ProductID']
['CustomerType', 'AgeGroup', 'Gender', 'Location', 'PaymentMethod', 'ProductName', 'Category', 'DiscountPercent']
['Quantity', 'UnitPrice', 'TotalAmount', 'FinalAmount', 'CostPrice', 'StockQuantity', 'ReorderLevel', 'SellingPrice']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Quantity       90000 non-null  int64  
 1   UnitPrice      90000 non-null  float64
 2   TotalAmount    90000 non-null  float64
 3   FinalAmount    90000 non-null  float64
 4   CostPrice      90000 non-null  float64
 5   StockQuantity  90000 non-null  int64  
 6   ReorderLevel   90000 non-null  int64  
 7   SellingPrice   90000 non-null  float64
dtypes: float64(5), int64(3)
memory usage: 5.5 MB


# IMPORTANT STEP FOR **SALES FORECASTING** - 'FinalAmount' is the target.

In [47]:
X = ml_model_df.drop('FinalAmount', axis=1).copy()
y = ml_model_df['FinalAmount'].copy()

# create a pipeline for preprocessing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

# impute for ml columns
pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median', add_indicator=True)),
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])

In [48]:
# fit the model
pipeline.fit(X_train, y_train)
# evaluate the model
score = pipeline.score(X_test, y_test)
print(f"Model R^2 Score: {score:.2f}")

Model R^2 Score: 0.97


In [49]:
# predict on the test set
y_pred = pipeline.predict(X_test)

# create a dataframe for predictions
predictions_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})
predictions_df.head()

,Actual,Predicted
58417,2686.59,2537.933307
12119,641.15,778.203668
15450,1169.09,1104.425089
53294,365.28,440.241077
49366,276.57,313.870812


In [50]:
# scores 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R^2 Score: {r2_score(y_test, y_pred):.2f}")


Mean Absolute Error: 154.37
Mean Squared Error: 55497.65
Root Mean Squared Error: 235.58
R^2 Score: 0.97


# CUSTOMER SEGMENTATION


In [51]:
ml_df.head()

,CustomerID,CustomerType,AgeGroup,Gender,Location,DateOfFirstPurchase,LastPurchaseDate,TransactionID,Date,ProductID,...,UnitPrice,TotalAmount,FinalAmount,ProductName,Category,CostPrice,StockQuantity,ReorderLevel,ExpiryDate,SellingPrice
0,C00000,Returning,18-25,Male,Lake Richardberg,2024-02-14,2024-06-07,T0020173,2025-02-14,P0064,...,260.28,780.84,780.84,Product Cohk,Wellness,181.79,99,26,2026-11-20,260.28
1,C00001,Returning,18-25,Other,Kellychester,2023-08-02,2024-07-12,T0022207,2025-05-05,P0805,...,510.17,4081.36,3469.16,Product jMJW,Equipment,436.77,319,42,2026-08-11,510.17
2,C00001,Returning,18-25,Other,Kellychester,2023-08-02,2024-07-12,T0033667,2024-11-08,P0394,...,490.52,4905.20,3678.90,Product zsBG,Wellness,337.36,343,43,2026-06-28,490.52
3,C00002,New,18-25,Female,Longbury,2024-03-29,2024-08-04,T0019925,2024-12-01,P0237,...,514.67,5146.70,4374.70,Product ipVZ,OTC,410.13,451,31,2026-04-03,514.67
4,C00002,New,18-25,Female,Longbury,2024-03-29,2024-08-04,T0025197,2024-08-04,P0663,...,533.44,2667.20,2533.84,Product usEA,OTC,374.32,423,36,2028-02-28,533.44


#### Necessary columns (we will use recency, frequency and monitary to find out about the customer segments):
* **CustomerID**	Unique identifier to group purchases
* **Date**	Date of each transaction (used to find recency + frequency)
* **TransactionID**	To count number of distinct purchases
* **FinalAmount**	To sum how much the customer spent
* **(Optional) LastPurchaseDate**	Can be used directly for recency if available

In [52]:
ml_df.columns

Index(['CustomerID', 'CustomerType', 'AgeGroup', 'Gender', 'Location',
       'DateOfFirstPurchase', 'LastPurchaseDate', 'TransactionID', 'Date',
       'ProductID', 'Quantity', 'DiscountPercent', 'PaymentMethod',
       'UnitPrice', 'TotalAmount', 'FinalAmount', 'ProductName', 'Category',
       'CostPrice', 'StockQuantity', 'ReorderLevel', 'ExpiryDate',
       'SellingPrice'],
      dtype='object')